<a href="https://colab.research.google.com/github/nixonleonard/ML-Simple-Chatbot-Project/blob/main/main_fin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tflearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import json, numpy as np, nltk
import random
import tensorflow as tf, tflearn
from nltk.stem.lancaster import *
stemmer = LancasterStemmer()

nltk.download('punkt')

Instructions for updating:
non-resource variables are not supported in the long term
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
from google.colab import files
upload = files.upload()

Saving dataset.json to dataset (2).json


#Importing Datasets

In [11]:
file_name = "dataset.json"
dataset = json.loads(upload[file_name].decode("utf-8"))
print(dataset)

{'dataset': [{'tag': 'greet', 'inputs': ['Hi', 'Is anyone there?', 'Hello', 'Morning', 'Afternoon', 'Evening', 'Good day', 'Whats up', 'Yoo', 'Hey', 'Nixobot'], 'responses': ['Hello!', 'Hi there, how can I help you?'], 'context_set': ''}, {'tag': 'status', 'inputs': ['How do you do?', 'How are you', 'How you feel'], 'responses': ["I'm fine thank you for asking..", "I'm good, and you?"], 'context_set': ''}, {'tag': 'status-Back', 'inputs': ['Im good', 'Im well', 'im not feeling well', 'not good'], 'responses': ['Is there anything that i can help you?'], 'context_set': ''}, {'tag': 'bye', 'inputs': ['See you later', 'Goodbye', 'I am Leaving', 'Have a Good day', 'Bye', 'Cya'], 'responses': ['Sad to see you go :(', 'Talk to you later', 'Goodbye!'], 'context_set': ''}, {'tag': 'age', 'inputs': ['how old', 'how old are you', 'what is your age', 'age?'], 'responses': ['I am 20 years old!', '... Im a bot you know ...'], 'context_set': ''}, {'tag': 'name', 'inputs': ['what is your name', 'what 

#Model

In [12]:
words = []
tags = []
input_dc = []
tag_dc = []

# Spit each sentesnce into single word with nltk, so we can see list of different words, 
# and extend to insert all the list without using loop
for data in dataset["dataset"]:
    for inp in data["inputs"]:
        wdtkn = nltk.word_tokenize(inp)
        words.extend(wdtkn)
        input_dc.append(wdtkn)
        tag_dc.append(data["tag"])
    if data["tag"] not in tags:
        tags.append(data["tag"])

# Stem word and remove duplicate elements
words = [stemmer.stem(w.lower()) for w in words if w != "?"]
words = sorted(list(set(words)))
tags = sorted(tags)

train = []
output = []

temp_out = [0 for _ in range(len(tags))] 

# Bag of word to see the frequency of each word occur (One Hot Encoded)
for x, dc in enumerate(input_dc):
    bag_of_words = []
    wrd = [stemmer.stem(w.lower()) for w in dc]
    
    for w in words:
        if w in wrd:
            bag_of_words.append(1)
        else:
            bag_of_words.append(0)
    
    output_row = temp_out[:]
    output_row[tags.index(tag_dc[x])] = 1

    train.append(bag_of_words)
    output.append(output_row)

train = np.array(train)
output = np.array(output)

##Deep Neural Network Model with Tflearn

In [13]:
nnm = tflearn.input_data(shape=[None, len(train[0])])   # Shape of input
nnm = tflearn.fully_connected(nnm, 8) # first hidden layer with 8 neurons
nnm = tflearn.fully_connected(nnm, 8) # second hidden layer with 8 neurons
nnm = tflearn.fully_connected(nnm, len(output[0]), activation="softmax") #activation function
nnm = tflearn.regression(nnm)

model = tflearn.DNN(nnm)

##Model Training / Fitting

In [7]:
model.fit(train, output, n_epoch=1000, batch_size=8, show_metric=True)

Training Step: 5999  | total loss: 0.05093 | time: 0.016s
| Adam | epoch: 1000 | loss: 0.05093 - acc: 1.0000 -- iter: 40/42
Training Step: 6000  | total loss: 0.04878 | time: 0.020s
| Adam | epoch: 1000 | loss: 0.04878 - acc: 1.0000 -- iter: 42/42
--


##Function to turn user input into bag of words

In [8]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
            
    return np.array(bag)

##Function to start chatting with the bot and the responses

In [9]:
def chatting():
    print("Start talking with the bot (type quit to stop)!")
    while True:
        inp = input("You: ")
        if inp.lower() == "exit":
            print('Thankyou for asking, i\'ll talk to you later :) !')
            break

        # Predict the response
        response = model.predict([bag_of_words(inp, words)]) [0]
        r_id = np.argmax(response)
        tag = tags[r_id]

        # Print the highest score, but if the score is more than 60%
        if response[r_id] > 0.6:
            for tg in dataset["dataset"]:
                if tg['tag'] == tag:
                    responses = tg['responses']

            print(random.choice(responses))
        else:
            print("Sorry i didn't get that, you can ask another question..")

#Let's Start Chatting

In [10]:
chatting()

Start talking with the bot (type quit to stop)!
You: Hey
Hi there, how can I help you?
You: How are you
I'm fine thank you for asking..
You: What should i call you
I'm Nixobot!
You: Can i ask how old are you
I am 20 years old!
You: so what menu do you recommend
We sell cakes for 3$ only!
You: and what is your operation time
Sorry i didn't get that, you can ask another question..
You: i mean hours operation time
We are open 7am-4pm Monday-Friday!
You: is there any discount?
Sorry, we don't have any discount for today :(
You: you have discount
Sorry, we don't have any discount for today :(
You: ok thankyou then
Sorry i didn't get that, you can ask another question..
You: exit
Thankyou for asking, i'll talk to you later :) !
